In [86]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data
    
def create_data_struct():
    modules = {}
    types = {}
    for line in get_data(20):
        key, val = line.split(' -> ')
        if key == 'broadcaster':
            type = 'b'
        else:
            type = key[0]
            key = key[1:]
        modules[key] = val.split(', ')
        types[key] = type
    return modules, types
        

def find_parent_state(child_state, modules, types):
    out = []
    for k,v in modules.items():
        if child_state in v:
            out.append((k, types[k]))
    return out
    


from collections import deque, defaultdict
from math import gcd

def lcm(xs):
  ans = 1
  for x in xs:
    ans = (ans*x)//gcd(x,ans)
  return ans

In [ ]:
modules, types = create_data_struct()


connector_memory = {}
INV = defaultdict(list)

for from_module, to_module_list in modules.items():
    for to_module in to_module_list:
        if (to_module in types) and (types[to_module]=='&'):
            if to_module not in connector_memory:
                connector_memory[to_module] = {}
            connector_memory[to_module][from_module] = False
        INV[to_module].append(from_module)


FROM = {}
INV = defaultdict(list)
for x,ys in R.items():
  R[x] = [adjust(y) for y in ys]
  for y in R[x]:
    if y[0]=='&':
      if y not in FROM:
        FROM[y] = {}
      FROM[y][x] = 'lo'
    INV[y].append(x)

assert len(INV['rx'])==1
assert INV['rx'][0][0]=='&'
WATCH = INV[INV['rx'][0]]

Q = deque()
PREV = {}
COUNT = defaultdict(int)
TO_LCM = []
ON = set()
hi = 0
lo = 0

for ii in range(10**10):
    state = ('broadcaster', 'button', False) # to from pulse
    Q.append(state)

    while Q:
        to_module, from_module, pulse  = Q.popleft()


        ### part2 hack
        if to_module in PREV and COUNT[to_module]==2 and to_module in WATCH:
            TO_LCM.append(ii-PREV[to_module])
        PREV[to_module] = t
        COUNT[to_module] += 1
        if len(TO_LCM) == len(WATCH):
            print(lcm(TO_LCM))
            sys.exit(0)
            
        if pulse:
            hi += 1
        else:
            lo += 1

        if to_module not in modules:
            continue
       
        if to_module=='broadcaster':
            for next_module in modules[to_module]:
                Q.append((next_module, to_module, pulse))
        
        elif types[to_module]=='%':
            if pulse:
                continue
            else:
                if to_module not in ON:
                    ON.add(to_module)
                    pulse = True
                else:
                    ON.discard(to_module)
                    pulse = False
                for next_module in modules[to_module]:
                    Q.append((next_module, to_module, pulse))
                    
        elif types[to_module] == '&':
            connector_memory[to_module][from_module] = pulse
            new_pulse  = (False if all(y==True for y in connector_memory[to_module].values()) else True)
            for next_module in modules[to_module]:
                Q.append((next_module, to_module, new_pulse))
        else:
             assert False, to_module
    if ii == 999:
        print(lo*hi)



711650489


In [81]:
import sys
import re
from copy import deepcopy
from math import gcd
from collections import defaultdict, Counter, deque
import heapq
import math
#D = open(sys.argv[1]).read().strip()
L = get_data(20)
G = [[c for c in row] for row in L]
R = len(G)
C = len(G[0])

def lcm(xs):
  ans = 1
  for x in xs:
    ans = (ans*x)//math.gcd(x,ans)
  return ans

TYP = {}

R = {}
for line in L:
  src, dest = line.split('->')
  src = src.strip()
  dest = dest.strip()
  dest = dest.split(', ')
  R[src] = dest
  TYP[src[1:]] = src[0]

def adjust(y):
  if y in TYP:
    return TYP[y]+y
  else:
    return y

FROM = {}
INV = defaultdict(list)
for x,ys in R.items():
  R[x] = [adjust(y) for y in ys]
  for y in R[x]:
    if y[0]=='&':
      if y not in FROM:
        FROM[y] = {}
      FROM[y][x] = 'lo'
    INV[y].append(x)

assert len(INV['rx'])==1
assert INV['rx'][0][0]=='&'
WATCH = INV[INV['rx'][0]]

lo = 0 
hi = 0
Q = deque()
ON = set()
PREV = {}
COUNT = defaultdict(int)
TO_LCM = []
#for t in range(1, 10**8):
for t in range(1, 1003):
  Q.append(('broadcaster', 'button', 'lo'))

  while Q:
    x, from_, typ = Q.popleft()
    #print(t-1, x, from_, typ)
#    print(x,from_,typ)
    if typ=='lo':
      # Assume &df -> rx AND &a -> df, &b -> df, etc.
      # We want to know when &a,&b get 'lo' input, because then they will send a 'hi' input to &df
      # When will &df get lo inputs from all of them?
      # Assume that they get lo inputs on a perfect cycle and that df must get a 'hi' input from all of them on the same
      #  timestamp. I assume they get a lot of 'hi' inputs on other timestamps that "reset" them?
      if x in PREV and COUNT[x]==2 and x in WATCH:
        #print(f't={t} cycle={t-PREV[x]} x={x} new_typ={new_typ}')
        TO_LCM.append(t-PREV[x])
      PREV[x] = t
      COUNT[x] += 1
    if len(TO_LCM) == len(WATCH):
      print(lcm(TO_LCM))
      sys.exit(0)

    if x=='rx' and typ=='lo':
      # won't happen; takes too long
      print(t+1)

    if typ=='lo':
      lo += 1
    else:
      hi += 1

    if x not in R:
      continue
    if x=='broadcaster':
      for y in R[x]:
        Q.append((y, x, typ))
    elif x[0]=='%':
      if typ=='hi':
        continue
      else:
        if x not in ON:
          ON.add(x)
          new_typ = 'hi'
        else:
          ON.discard(x)
          new_typ = 'lo'
        for y in R[x]:
          Q.append((y, x, new_typ))
         
    elif x[0]=='&':
      FROM[x][from_] = typ
      new_typ = ('lo' if all(y=='hi' for y in FROM[x].values()) else 'hi')
      for y in R[x]:
        Q.append((y, x, new_typ))
    else:
      assert False, x
  print(hi, lo)
  if t==1000:
    print(lo*hi)

36 9
68 26
104 35
153 58
189 67
221 84
257 93
327 125
363 134
395 151
431 160
480 183
516 192
548 209
584 218
666 259
702 268
734 285
770 294
819 317
855 326
887 343
923 352
993 384
1029 393
1061 410
1097 419
1146 442
1182 451
1214 468
1250 477
1354 527
1390 536
1422 553
1458 562
1507 585
1543 594
1575 611
1611 620
1681 652
1717 661
1749 678
1785 687
1834 710
1870 719
1902 736
1938 745
2020 786
2056 795
2088 812
2124 821
2173 844
2209 853
2241 870
2277 879
2347 911
2383 920
2415 937
2451 946
2500 969
2536 978
2568 995
2604 1004
2720 1063
2756 1072
2788 1089
2824 1098
2873 1121
2909 1130
2941 1147
2977 1156
3047 1188
3083 1197
3115 1214
3151 1223
3200 1246
3236 1255
3268 1272
3304 1281
3386 1322
3422 1331
3454 1348
3490 1357
3539 1380
3575 1389
3607 1406
3643 1415
3713 1447
3749 1456
3781 1473
3817 1482
3866 1505
3902 1514
3934 1531
3970 1540
4074 1590
4110 1599
4142 1616
4178 1625
4227 1648
4263 1657
4295 1674
4331 1683
4401 1715
4437 1724
4469 1741
4505 1750
4554 1773
4590 1782
4622 1

In [ ]:
        elif types[to_module] == '&':
            connector_memory[to_module][from_module] = pulse
            new_pulse  = (False if all(y==True for y in connector_memory[to_module].values()) else True)
            for next_module in modules[to_module]:
                Q.append((next_module, to_module, new_pulse))
                if next_module=='qz' and to_module=='xd' and ii<2:  
                    print(ii, next_module, to_module, pulse, new_pulse, connector_memory[to_module])
